In [1]:
import os
HOME = os.getcwd()
print(HOME)

/kaggle/working


In [2]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.20 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5519.7/8062.4 GB disk)


In [3]:
from ultralytics import YOLO
from IPython.display import display, Image

# Detection with pre-trained model

In [4]:
%cd {HOME}
!yolo task=detect mode=predict model=yolov8n.pt conf=0.25 source='https://media.roboflow.com/notebooks/examples/dog.jpeg' save=True

/kaggle/working
100%|██████████████████████████████████████| 6.23M/6.23M [00:00<00:00, 60.3MB/s]

2024-03-25 16:32:40.887615: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-25 16:32:40.887728: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-25 16:32:41.048871: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Ultralytics YOLOv8.0.20 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs
100%|████████████████████████████████████████| 104k/104k [00:00<00:00, 51.1MB/s]
image 1/1 /kaggle/workin

# Import datasets from roboflow

In [5]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

/kaggle/working/datasets


In [6]:
%cd {HOME}/datasets
!pip install roboflow
from roboflow import Roboflow

rf = Roboflow(api_key="A88aglIXim3OCSF5vNOe")
project = rf.workspace("gabriel-advent-hobto").project("detect-tdz36")
version = project.version(1)
dataset = version.download("yolov8")


/kaggle/working/datasets
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 24.7 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.9.0.80
    Uninstalling opencv-python-headless-4.9.0.80:
      Successfully uninstalled opencv-python-headless-4.9.0.80
  Attempting uninstall: idna
    Found existing installation: idna 3.6
    Uninstalling idna-3.6:
      Successfully uninstalled idna-3.6
  Attempting uninstall: cycler
    Found existing installation: cycler 0.12.1
    Uninstalling cycler-0.12.1:
      Successfully uninstalled cycler-0.12.1
  Attempting uninstall: certifi
    Found existing installatio


Extracting Dataset Version Zip to detect-2 in yolov8:: 100%|██████████| 6822/6822 [00:00<00:00, 8372.01it/s]


# Train my dataset without tuning

In [7]:
%cd {HOME}

data_yaml='/kaggle/working/datasets/detect-1/data.yaml'

!yolo task=detect mode=train model=yolov8m.pt data=data_yaml epochs=200 imgsz=416 plots=True

# Train my dataset with tuning

In [8]:
%cd {HOME}
!yolo detect train \
    data='/kaggle/working/datasets/detect-2/data.yaml' \
    model=yolov8s.pt \
    epochs=200 \
    imgsz=640 \
    optimizer='Adam' \
    lr0=0.0001 \
    batch=16 \
    plots=True \
    weight_decay=0.0005 \
    momentum=0.937 \
    save=True

/kaggle/working
100%|███████████████████████████████████████| 21.5M/21.5M [00:00<00:00, 151MB/s]

Ultralytics YOLOv8.0.20 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8s.yaml, data=/kaggle/working/datasets/detect-2/data.yaml, epochs=200, patience=50, batch=16, imgsz=640, save=True, cache=False, device=, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=False, val=True, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=ultralytics/assets/, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=Fa

# Validasi Data

In [9]:
%cd {HOME}

!yolo task=detect mode=val model='/kaggle/working/runs/detect/train/weights/best.pt' data='/kaggle/working/datasets/detect-2/data.yaml' plots=True save_json=True

/kaggle/working
2024-03-25 18:25:40.532113: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-25 18:25:40.532167: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-25 18:25:40.533738: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Ultralytics YOLOv8.0.20 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11127906 parameters, 0 gradients, 28.4 GFLOPs
val: Scanning /kaggle/working/datasets/detect-2/valid/labels.cache... 284 images
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 91, len(boxes) = 584. To resolv

# Export model

In [10]:
#%cd {HOME}
#!yolo export model='/kaggle/working/runs/detect/train3/weights/best.pt'

In [11]:
#%cd {HOME}
#!yolo export model='/kaggle/working/runs/detect/train3/weights/last.pt'

# grid search

In [ ]:
import subprocess
from itertools import product

In [ ]:
param_grid = {
    'model': ['yolov8s.pt'],
    'batch': [32, 64],
    'dropout': [0.2, 0.5],
    'optimizer' : ['adam', 'RMSProp'],
    'lr' : [0.001, 0.0001]
}


# Generate semua kombinasi
param_combinations = list(product(*param_grid.values()))

_result = {}

# Loop over the grid of hyperparameters
for params in param_combinations:
    model, optimizer, batch, lr, dropout = params
    command = f"yolo detect train \
data='/kaggle/working/datasets/detect-2/data.yaml' \
model='{model}' \
epochs=10 \
imgsz=416 \
optimizer='{optimizer} \
lr0=0.0001 \
batch='{batch}' \
plots=True \
weight_decay=0.0005 \
dropout='{dropout}' \
save=True"

    # Run the command and get the validation loss
    results = subprocess.run(command, shell=True, capture_output=True, text=True)
    _result[params] = results['val/mAP_0.5']

best_params = max(_result, key=_result.get)
print(f"Best parameters: {best_params} with mAP: {_result[best_params]}")

# grid search 2

In [ ]:
    %cd {HOME}

    import subprocess
    from itertools import product

    param_grid = {
        'model': ['yolov8n.pt', 'yolov8s.pt', 'yolov8m.pt'],
        'imgsz': [240, 256, 416],
        'batch': [16, 32, 64],
        'momentum': [0.937, 0.8],
        'optimizer' : ['adam', 'adamW'],
    }

    # Generate semua kombinasi
    param_combinations = list(product(*param_grid.values()))

In [ ]:
%cd {HOME}
# Memasukan semua kombinasi parameter ke dalam commands
 
commands = []
for params in param_combinations:
    model, imgsz, batch, momentum, optimizer = params
    command = f"yolo detect train \
data='/kaggle/working/datasets/detect-2/data.yaml' \
model='{model}' \
epochs=200 \
imgsz='{imgsz}' \
optimizer={optimizer} \
lr0=0.0001 \
batch='{batch}' \
plots=True \
weight_decay=0.0005 \
momentum='{momentum}' \
save=True"
    commands.append(command)

In [ ]:
%cd {HOME}

import subprocess

# Loop setiap perintah dan jalankan menggunakan subprocess
for command in commands:
    print(command)
    subprocess.run(command, shell=True)

In [ ]:
%cd {HOME}
!yolo detect train \
    data='/kaggle/working/datasets/detect-2/data.yaml' \
    model=yolov8m.pt \
    epochs=200 \
    imgsz=240 \
    optimizer='Adam' \
    lr0=0.0001 \
    batch=64 \
    plots=True \
    weight_decay=0.0005 \
    momentum=0.937 \
    save=True